In [1]:
from pandas import *
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import daniel_utils as du
import datetime
%load_ext autoreload
%autoreload 2


/home/daniel/miniconda3/lib/python3.5/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [2]:
df = read_csv(filepath_or_buffer="csv/london.csv", header=0)

df = du.zero_to_nan(df)

df = df.dropna(subset={'L_PREOV', 'L_PERIOD'})

df = du.rename_it_en(df)
df = df.reset_index(drop=True)



In [3]:
for i in range(0, 200): # len(df) - 1
    #print(str(df['BEGIN_DATE'][i])[-2:])
    df['BEGIN_DATE'][i] = str(df['BEGIN_DATE'][i])[:-2] + "19" + str(df['BEGIN_DATE'][i])[-2:]
#print(df['BEGIN_DATE'])
df = df[df['ID'] == 9]
df = df.reset_index(drop=True)
df['BEGIN_DATE'] = pd.to_datetime(df['BEGIN_DATE'], format='%d-%m-%Y')

/home/daniel/miniconda3/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [4]:
curr_id = df['ID'][0]
n_cycles = df['N_CYCLES'][0]
print(curr_id)
print(n_cycles)
print(len(df['BEGIN_DATE']))
df_subject = pd.DataFrame(columns=['day', 'BBT'])

for x in range(0, n_cycles):
    print(x)
    #print(df['BEGIN_DATE'][x])
    #print(df['L_CYCLE'][x])
    #print(df['BEGIN_DATE'][x] + datetime.timedelta(days=int(df['L_CYCLE'][x])))
    df_cycle = du.get_cycle_with_bbt(df, x)
    
    # iterate through df_cycle, converting the day to the date on which the BBT was observed
    for i in range(0, len(df_cycle)):
        
        df_cycle['day'][i] = (df['BEGIN_DATE'][x] + datetime.timedelta(days = i))

   
    frames = [df_subject, df_cycle]
    df_subject = pd.concat(frames, ignore_index=True)

#for x in range(0, len(df_subject)):
#    print(str(df_subject['day'][x]) + " " + str(df_subject['BBT'][x]))

9
15
15
0


/home/daniel/miniconda3/lib/python3.5/site-packages/ipykernel/__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


1
2
3
4
5
6
7
8
9
10
11
12
13
14


In [5]:
for x in range(0, len(df_subject)):
    print(df_subject['day'][x])
    print(df_subject['BBT'][x])
    
    # next step: figure out how to do the discontinuities (loop thru every day btwn first begin date and last end date, and if no data, then just put nan)

1964-02-16 00:00:00
83.0
1964-02-17 00:00:00
80.0
1964-02-18 00:00:00
80.0
1964-02-19 00:00:00
80.0
1964-02-20 00:00:00
82.0
1964-02-21 00:00:00
80.0
1964-02-22 00:00:00
80.0
1964-02-23 00:00:00
81.0
1964-02-24 00:00:00
79.0
1964-02-25 00:00:00
80.0
1964-02-26 00:00:00
80.0
1964-02-27 00:00:00
80.0
1964-02-28 00:00:00
80.0
1964-02-29 00:00:00
83.0
1964-03-01 00:00:00
83.0
1964-03-02 00:00:00
83.0
1964-03-03 00:00:00
78.0
1964-03-04 00:00:00
81.0
1964-03-05 00:00:00
82.0
1964-03-06 00:00:00
84.0
1964-03-07 00:00:00
88.0
1964-03-08 00:00:00
88.0
1964-03-09 00:00:00
86.0
1964-03-10 00:00:00
85.0
1964-03-11 00:00:00
88.0
1964-03-12 00:00:00
86.0
1964-03-13 00:00:00
84.0
1964-03-14 00:00:00
86.0
1964-03-15 00:00:00
86.0
1964-03-16 00:00:00
83.0
1964-03-17 00:00:00
80.0
1964-03-18 00:00:00
80.0
1964-03-19 00:00:00
82.0
1964-03-20 00:00:00
81.0
1964-03-21 00:00:00
82.0
1964-03-22 00:00:00
81.0
1964-03-23 00:00:00
80.0
1964-03-24 00:00:00
80.0
1964-03-25 00:00:00
80.0
1964-03-26 00:00:00
78.0


In [6]:
total_days = df_subject['day'][len(df_subject)-1] - df_subject['day'][0]



for x in range(1, total_days.days):
    curr_date = df_subject['day'][x-1]
    prev_date = df_subject['day'][x]
    if(curr_date  + datetime.timedelta(days=1) != prev_date):
        print('discont!' + str(df_subject['day'][x-1]))
        line = DataFrame({"day": curr_date + datetime.timedelta(days=1), "BBT": np.nan}, index=[x])
        df_subject = concat([df_subject.ix[:x-1], line, df_subject.ix[x:]]).reset_index(drop=True)
        #print(df_subject)
        
df_subject.to_csv('csv/temp.csv')

discont!1965-02-28 00:00:00
discont!1965-03-01 00:00:00
discont!1965-03-02 00:00:00
discont!1965-03-03 00:00:00
discont!1965-03-04 00:00:00
discont!1965-03-05 00:00:00
discont!1965-03-06 00:00:00
discont!1965-03-07 00:00:00
discont!1965-03-08 00:00:00
discont!1965-03-09 00:00:00
discont!1965-03-10 00:00:00
discont!1965-03-11 00:00:00
discont!1965-03-12 00:00:00
discont!1965-03-13 00:00:00
discont!1965-03-14 00:00:00
discont!1965-03-15 00:00:00
discont!1965-03-16 00:00:00
discont!1965-03-17 00:00:00
discont!1965-03-18 00:00:00
discont!1965-03-19 00:00:00
discont!1965-03-20 00:00:00
discont!1965-03-21 00:00:00
discont!1965-03-22 00:00:00
discont!1965-03-23 00:00:00
discont!1965-03-24 00:00:00
discont!1965-03-25 00:00:00
discont!1965-03-26 00:00:00
discont!1965-03-27 00:00:00


In [7]:
%matplotlib --list
plt.figure(figsize=(100, 7))
plt.plot(df_subject['day'], df_subject['BBT'])
for index, row in df.iterrows():
    plt.axvline(row['BEGIN_DATE'] + datetime.timedelta(days = row['L_PERIOD']), color='red')
    plt.axvline(row['BEGIN_DATE'] + datetime.timedelta(days = row['L_PREOVULATION']), color='green')
    plt.axvline(row['BEGIN_DATE'], color='blue')
#plt.scatter(df_subject['day'], df_subject['BBT'])
plt.show()

Available matplotlib backends: ['nbagg', 'gtk3', 'qt5', 'qt', 'tk', 'wx', 'osx', 'gtk', 'notebook', 'qt4', 'inline']
